In [1]:
import datetime

from rocketpy import Environment

env = Environment(latitude=42.692624, longitude=-77.18899, elevation=2333)

tomorrow = datetime.date.today() + datetime.timedelta(days=1)

env.set_date(
    (tomorrow.year, tomorrow.month, tomorrow.day, 12)
)  # Hour given in UTC time

env.set_atmospheric_model(
    type="Forecast", file="GFS"  # "custom_atmosphere", wind_v, wind_u
)

In [ ]:
env.info()

In [2]:
from rocketpy.motors import SolidMotor

solid_motor = SolidMotor(
    thrust_source="Curves/enterprise_thrust.csv",
    dry_mass=4.559,
    dry_inertia=(0.0996, 0.0996, 0.00307),
    nozzle_radius=0.0397,
    grain_number=1,
    grain_density=1807,
    grain_outer_radius=0.0397,
    grain_initial_inner_radius=0.0381,
    grain_initial_height=0.508,
    grain_separation=0,
    grains_center_of_mass_position=0.52,
    center_of_dry_mass_position=1.33,
    nozzle_position=2.22,
    # burn_time=2.764,
    throat_radius=0.0397,
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)

In [ ]:
solid_motor.info()

In [3]:
from rocketpy import Rocket

rocket = Rocket(
    radius=15.7 / 200,
    mass=18.2,  # rocket's mass without the motor in kg
    inertia=(
        7.503,
        7.503,
        0.05608,
    ),  # in relation to the rocket's center of mass without motor
    power_off_drag="Curves/enterprise_drag.csv",
    power_on_drag="Curves/enterprise_drag.csv",
    center_of_mass_without_motor=1.34,
    coordinate_system_orientation="tail_to_nose",
)

In [ ]:
motor = solid_motor

rocket.add_motor(motor, position = 0.012)

rail_buttons = rocket.set_rail_buttons(
    upper_button_position=0.61,
    lower_button_position=0.00,
    angular_position=180,
)

nose_cone = rocket.add_nose(length=0.657, kind="von karman", position=2.22)

fin_set = rocket.add_trapezoidal_fins(
    n=3,
    root_chord=0.41,
    tip_chord=0.05,
    span=0.165,
    cant_angle=0.0,
    # airfoil=("Airfoils/Airfoil.csv", "radians"),
    position=0.45,
    sweep_length=0.354,
    # sweep_angle=57.9,
)

tail = rocket.add_tail(
    top_radius=0.0785, bottom_radius=0.04635, length=0.0356, position=0.03
)

main = rocket.add_parachute(
    name="main",
    cd_s=2.200,
    trigger=305,  # ejection altitude in meters
    sampling_rate=105,
    lag=0,
    noise=(0, 8.3, 0.5),
)

drogue = rocket.add_parachute(
    name="drogue",
    cd_s=0.800,
    trigger="apogee",  # ejection at apogee
    sampling_rate=105,
    lag=0,
    noise=(0, 8.3, 0.5),
)

In [ ]:
rocket.info()

In [ ]:
rocket.draw()

In [ ]:
rocket.plots.static_margin()

In [7]:
from rocketpy import Flight

test_flight = Flight(
    rocket=rocket,
    environment=env,
    rail_length=4,  # metres
    inclination=90,  # degrees
    heading=0,  # degrees
)

In [ ]:
test_flight.info()

In [ ]:
test_flight.export_kml("trajectory.kml")

In [ ]:
from rocketpy.simulation import MonteCarlo
from rocketpy.stochastic import (
    StochasticEnvironment,
    StochasticSolidMotor,
    StochasticRocket,
    StochasticFlight,
    StochasticNoseCone,
    StochasticTail,
    StochasticTrapezoidalFins,
    StochasticParachute,
    StochasticRailButtons,
)

""" All stochastics values can be lists of possible values or have the form of:
(nominal value[optional], standard deviation, distribution type[optional])
type = “normal”, “binomial”, “chisquare”, “exponential”, “gamma”, “gumbel”, “laplace”, “logistic”, “poisson”, “uniform”, and “wald” """

stochastic_env = StochasticEnvironment(
    environment=env,
    longitude=0.01,
    latitude=0.01,
)

stochastic_motor = StochasticSolidMotor(
    solid_motor=solid_motor,
    burn_start_time=(0, 0.1, "binomial"),
    total_impulse=100,
    nozzle_radius=0.0001,
    nozzle_position=0.01,
)

stochastic_rocket = StochasticRocket(
    rocket=rocket,
    radius=0.0001,
    mass=(0.5, "normal"),
    inertia_11=0.1,
    inertia_22=0.1,
    inertia_33=0.001,
    center_of_mass_without_motor=0,
)

stochastic_rail_buttons = StochasticRailButtons(
    rail_buttons=rail_buttons, buttons_distance=0.01
)

stochastic_nose_cone = StochasticNoseCone(
    nosecone=nose_cone, length=0.001
)

stochastic_fin_set = StochasticTrapezoidalFins(
    trapezoidal_fins=fin_set,
    root_chord=0.01,
    tip_chord=0.01,
    span=0.001,
)

stochastic_tail = StochasticTail(
    tail=tail, top_radius=0.0001, bottom_radius=0.0001, length=0.0001
)

stochastic_main = StochasticParachute(parachute=main, cd_s=0.1, lag=0.1)

stochastic_drogue = StochasticParachute(
    parachute=drogue, cd_s=0.07, lag=0.1
)

stochastic_rocket.add_motor(stochastic_motor, position=0.001)

stochastic_rocket.set_rail_buttons(
    stochastic_rail_buttons, lower_button_position=(0.01, "normal")
)

stochastic_rocket.add_nose(stochastic_nose_cone, position=0.01)

stochastic_rocket.add_trapezoidal_fins(stochastic_fin_set, position=(0.01, "normal"))

stochastic_rocket.add_tail(stochastic_tail)

stochastic_rocket.add_parachute(stochastic_main)

stochastic_rocket.add_parachute(stochastic_drogue)

stochastic_flight = StochasticFlight(
    flight=test_flight,
    inclination=(88, 1),  # mean= 88, std=1
    heading=(0, 2),  # mean= 0, std=2
)

test_dispersion = MonteCarlo(
    filename="Monte_Carlo_results/monte_carlos_analysis_results",
    environment=stochastic_env,
    rocket=stochastic_rocket,
    flight=stochastic_flight,
)

C:\Users\arthu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\rocketpy\simulation\monte_carlo.py:133: UserWarning: This class is still under testing and some attributes may be changed in next versions
  warnings.warn(


In [17]:
stochastic_env.visualize_attributes()

Reporting the attributes of the `StochasticEnvironment` object:

Constant Attributes:
	datum                    SIRGAS2000
	elevation                344.6397705078125
	gravity                  Function from R1 to R1 : (height (m)) → (gravity (m/s²))
	timezone                 UTC

Stochastic Attributes:
	latitude                 42.69262 ± 0.01000 (normal)
	longitude                -77.18899 ± 0.01000 (normal)
	wind_velocity_x_factor   1.00000 ± 0.00000 (normal)
	wind_velocity_y_factor   1.00000 ± 0.00000 (normal)


In [18]:
stochastic_motor.visualize_attributes()

Reporting the attributes of the `StochasticSolidMotor` object:

Constant Attributes:
	burn_out_time                    2.764
	center_of_dry_mass_position      1.33
	coordinate_system_orientation    nozzle_to_combustion_chamber
	dry_I_11                         0.0996
	dry_I_12                         0
	dry_I_13                         0
	dry_I_22                         0.0996
	dry_I_23                         0
	dry_I_33                         0.00307
	dry_mass                         4.559
	grain_density                    1807
	grain_initial_height             0.508
	grain_initial_inner_radius       0.0381
	grain_number                     1
	grain_outer_radius               0.0397
	grain_separation                 0
	grains_center_of_mass_position   0.52
	interpolate                      linear
	throat_radius                    0.0397
	thrust_source                    Curves/enterprise_thrust.csv

Stochastic Attributes:
	burn_start_time                  0.00000 ± 0.10000 (binomia

In [19]:
stochastic_rocket.visualize_attributes()

Reporting the attributes of the `StochasticRocket` object:

Constant Attributes:
	I_12_without_motor              0
	I_13_without_motor              0
	I_23_without_motor              0
	coordinate_system_orientation   tail_to_nose
	power_off_drag                  Function from R1 to R1 : (Mach Number) → (Drag Coefficient with Power Off)
	power_on_drag                   Function from R1 to R1 : (Mach Number) → (Drag Coefficient with Power On)

Stochastic Attributes:
	I_11_without_motor              6.32100 ± 0.00000 (normal)
	I_22_without_motor              7.50300 ± 0.01000 (normal)
	I_33_without_motor              0.05608 ± 0.01000 (normal)
	center_of_mass_without_motor    1.34000 ± 0.00000 (normal)
	mass                            18.20000 ± 0.50000 (normal)
	power_off_drag_factor           1.00000 ± 0.00000 (normal)
	power_on_drag_factor            1.00000 ± 0.00000 (normal)
	radius                          0.07850 ± 0.00001 (normal)


In [20]:
stochastic_flight.visualize_attributes()

Reporting the attributes of the `StochasticFlight` object:

Constant Attributes:
	rail_length           4

Stochastic Attributes:
	heading               53.00000 ± 2.00000 (normal)
	inclination           84.70000 ± 1.00000 (normal)


In [9]:
test_dispersion.simulate(number_of_simulations=10, append=True)

Completed 10 iterations. Total CPU time: 12.5 s. Total wall time: 13.5 sted time left: 0 s 
Saving results.   
Results saved to Monte_Carlo_results/monte_carlos_analysis_results.outputs.txt


In [10]:
test_dispersion.set_results()

In [11]:
test_dispersion.set_processed_results()

In [12]:
test_dispersion.prints.all()

Monte Carlo Simulation by RocketPy
Data Source:  Monte_Carlo_results/monte_carlos_analysis_results
Number of simulations:  10
Results: 

                Parameter            Mean          Median       Std. Dev.    95% PI Lower    95% PI Upper
--------------------------------------------------------------------------------------------------------------
              apogee_time          21.399          21.431           0.203          21.068          21.737
                   apogee        2633.069        2642.332          53.021        2544.640        2714.024
     frontal_surface_wind           2.120           2.096           0.127           1.917           2.355
out_of_rail_stability_margin           5.727           5.727           0.019           5.698           5.753
 initial_stability_margin           5.719           5.719           0.019           5.690           5.745
                  t_final         138.081         138.686           2.909         132.187         141.389
       

In [ ]:
test_dispersion.plots.all()

In [16]:
test_dispersion.export_ellipses_to_kml(
    filename="Monte_Carlo_results/monte_carlo_class_example.kml",
    origin_lon=-77.18899,
    origin_lat=42.692624,
    type="all",
)